# dual guide parser - August 2021
 - **Project:** iNDI.
 - **Author(s):** Dan Ramos, Lirong Peng, Faraz Faghri and Mike Nalls 

---
### Quick Description: 
- **Problem:** We need a method that preprocesses guides for experiments, something that parses fastqs to only include those guides included in R1 and R1 from the concensus guide list ... we also need to catalogue failed reads. We also need this to take MiSeq and other data.
- **Solution:** The workflow below sums it up pretty well. Let's test out some code on small iNDI datasets provided by Dan R. We have added support from SeqIO 

### Workflow:
0.   Set up notebook.
1.   Import data, this includes concensus guides and R1 + R2 fastqs.
2.   Identify matching read groups across R1 and R2.
3.   Reduce the datasets to the read groups that match in R1 and R2.
4.   Split into 'hits.\*'  and 'recombinants.\*'.  
'hits.\*' denotes read group matches and the protospacers match.
'recombinants.\*' denotes read group matches but one or more protospacers does not.
5.   Export 'hits.\*' and 'recombinants.\*' per fastq.

### Notes on data for testing:
- **20200513_library_1_2_unbalanced_dJR051.csv** = All elements of the dual sgRNA library. Sequence from protospacer_A and protospacer_B columns must be present in the same row to be considered a match.  
- **UDP0011_S5_R1_001.fastq.gz** = First 20 bases of each read should match "protospacer_A" sequence from "20200513_library_1_2_unbalanced_dJR051.csv".  
- **UDP0011_S5_R2_001.fastq.gz** = First 20 bases of each read should match reverse complement of "protospacer_B" sequence from "20200513_library_1_2_unbalanced_dJR051.csv".  
** The major change in V3 of this code is matching sequences for the guides using all UPPER CASE bases intead of being case-sensitive."
** V5 skips the first base since it is always G. Essentially running 19bp matches.



# 0.   Set up notebook.

In [ ]:
import os
# from google.colab import drive
import pandas.util.testing as tm
import h5py
import numpy as np
import pandas as pd
import math
import sys
import joblib
import subprocess
import argparse
import gzip
import textwrap

# !pip install --upgrade tables
# ! pip install biopython

# import tables

import requests
from Bio import SeqIO 
from Bio.Seq import reverse_complement
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from itertools import islice
# Comment out below after testing.

# drive.mount('/content/drive/')
# os.chdir("/content/drive/Shared drives/CARD_iNDI/scratch/dual_guide_parser")
# ! pwd


Mounted at /content/drive/
/content/drive/Shared drives/CARD_iNDI/scratch/dual_guide_parser


In [ ]:
# Set  options for testing.

# guides_file = "20200513_library_1_2_unbalanced_dJR051.csv"
# r1_file = "UDP0011_S5_R1_001.fastq.gz"
# r2_file = "UDP0011_S5_R2_001.fastq.gz"
# N_rows = 10000 # Speed up testing, this just reads the first 10K sequences.

# Set the options for production.

parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, description=textwrap.dedent('''\

Thanks for trying the dual_guide_parser from CARD + iNDI + DTi.
To run this code, you will need to specify the guides_file, this is a file similar to the example 20200513_library_1_2_unbalanced_dJR051.csv.
You will need to specify a pair of R1 and R2 files, such as UDP0007_S1_R1_001.fastq.gz and UDP0007_S1_R2_001.fastq.gz.
You can also specify the number of read groups you are interested for testing the tool, this relates to the option n_groups. 
This will only read that many readgroups from the R1 and R2 files, allowing you to speed things up a bit.
This code must be run from the working directory that contains the R1 and R2 files, but the guides_file can be anywhere, just 
specify a full path to the guides file like ~/Desktop/20200513_library_1_2_unbalanced_dJR051.csv.
This is best run on a large RAM / high CPU set up as the files are quite large.
Finally, to run this code, you will need several packages, including biopython. To see the required packages listed, run with the -h option.

'''))    
parser.add_argument('--packages', help='Request for packages required to run, and how to install.', action='store_true')
parser.add_argument('--guides_file', type=str, default='missing', help='Mandatory input filepath. This is a file similar to the example 20200513_library_1_2_unbalanced_dJR051.csv. This can be a complete filepath')
parser.add_argument('--r1_file', type=str, default='missing', help='Mandatory input file name. An R1 file in your working directory.')
parser.add_argument('--r2_file', type=str, default='missing', help='Mandatory input file name. An R2 file in your working directory.')
parser.add_argument('--N_reads', type=int, default=0, help='Optional number of readgroups to test. An integer.')

args = parser.parse_args()

if(args.packages):
  print("Must have numpy and pandas available. \n Additionally, must install biopython.")
  print("To install biopython, run pip install biopython, or, if using conda,")
  print("conda install -c conda-forge biopython")
  quit()
 
print("#"*46)
print("")
print("Here is some basic info on the command you are about to run.")
print("Python version info...")
print(sys.version)
print("CLI argument info...")
print("The guides file you are using is", args.guides_file, ".")
print("The r1 file you are using is", args.r1_file, ".")
print("The r2 file you are using is", args.r2_file, ".")
print("How many read groups are only for a quick test and not the full set?", args.N_reads, ".")
print("")
print("#"*46)

guides_file = args.guides_file
r1_file = args.r1_file
r2_file = args.r2_file
N_reads = args.N_reads
N_rows = N_reads

##############################################
Thanks for trying the dual_guide_parser from CARD + iNDI + DTi.
To run this code, you will need to specify the guides_file, this is a file similar to the example 20200513_library_1_2_unbalanced_dJR051.csv.
You will need to specify a pair of R1 and R2 files, such as UDP0007_S1_R1_001.fastq.gz and UDP0007_S1_R2_001.fastq.gz.
You can also specify the number of read groups you are interested for testing the tool, this relates to the option n_groups. This will only read that many readgroups from the R1 and R2 files, allowing you to speed things up a bit.
This code must be run from the working directory that contains the R1 and R2 files, but the guides_file can be anywhere, just specify a full path to the guides file like ~/Desktop/20200513_library_1_2_unbalanced_dJR051.csv.
This is best run on a large RAM / high CPU set up as the files are quite large.
Finally, to run this code, you will need several packages, including biopython. To see the re

usage: ipykernel_launcher.py [-h] [--guides_file GUIDES_FILE]
                             [--r1_file R1_FILE] [--r2_file R2_FILE]
                             [--N_reads N_READS]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-b6010eba-8738-4fa4-9ee7-973bb74b8ff7.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
%tb

SystemExit: ignored

# 1. Import data, this includes concensus guides and R1 + R2 fastqs.

In [ ]:
# Import data to pandas.
guides_df = pd.read_csv(guides_file, engine='c')
# 
# # Import R1s and R2s.
# ## pysam way Pysam introduces many other file format compatibilities such as 
# ## CRAM/SAM/BAM
# if (N_rows == 0):
#   with pysam.FastxFile(r1_file) as fh:
#     r1_df = pd.DataFrame([(entry.name, entry.sequence, entry.comment, entry.quality) for entry in fh], columns=['name','seq', 'comment', 'qual'])
#   with pysam.FastxFile(r2_file) as fh:
#     r2_df = pd.DataFrame([(entry.name, entry.sequence, entry.comment, entry.quality) for entry in fh], columns=['name','seq', 'comment', 'qual'])
# else:
#   with pysam.FastxFile(r1_file) as fh:
#     r1_df = pd.DataFrame([(entry.name, entry.sequence, entry.comment, entry.quality) for entry in islice(fh,0,N_rows)], columns=['name','seq', 'comment', 'qual'])
#   with pysam.FastxFile(r2_file) as fh:
#     r2_df = pd.DataFrame([(entry.name, entry.sequence, entry.comment, entry.quality) for entry in islice(fh,0,N_rows)], columns=['name','seq', 'comment', 'qual'])
# 
# ## SeqIO way
# For more compatitibility with other files types will need to import as SeqIO objects.
# Consider the following suggestions if so.
# use to_dict for compatibility with more file types and for true dictionary
# functionality. If files too big, use .index.
# Otherwise, if more memory needed, instantiate as a list
with gzip.open(r1_file, mode = 'rt') as r1, gzip.open(r2_file, mode = 'rt') as r2: 
  r1_it = FastqGeneralIterator(r1)
  r2_it = FastqGeneralIterator(r2)

  if (N_rows == 0):
    r1_df = pd.DataFrame(r1_it, columns=['title', 'seq', 'qual'])
    r2_df = pd.DataFrame(r2_it, columns=['title', 'seq', 'qual'])
  else:
    r1_df = pd.DataFrame(islice(r1_it, 0, N_rows), columns=['title', 'seq', 'qual'])
    r2_df = pd.DataFrame(islice(r2_it, 0, N_rows), columns=['title', 'seq', 'qual'])

In [ ]:
# only do this once
r1_df.insert(loc=2, column='plus', value='+')
r2_df.insert(loc=2, column='plus', value='+')

def split_str(s: str) -> str:
    return s.split(" ", maxsplit = 1)[0]
r1_df['read_group'] = r1_df["title"].apply(split_str)
r2_df['read_group'] = r2_df["title"].apply(split_str)

r1_df['title'] = '@' + r1_df['title']
r2_df['title'] = '@' + r2_df['title']

This next code block defines the r1 and r2 keys that are 19 BP, also making a reverse complement of r2.

In [ ]:
# New as of v5.

guides_df['protospacer_A_19bp_trimmed'] = [x[1:20] for x in guides_df['protospacer_A']]
guides_df['protospacer_B_19bp_trimmed'] = [x[1:20] for x in guides_df['protospacer_B']]

# Make guide key columns.

guides_df['r1_key'] = guides_df['protospacer_A_19bp_trimmed']

# R2 is tricky as it is the reverse compliment. Flip it and translate using the function below.

guides_df['r2_key'] = guides_df['protospacer_B_19bp_trimmed'].apply(reverse_complement)
guides_df['r1_r2_key'] = guides_df['r1_key'] + "_" + guides_df['r2_key']

# 2. Identify matching read groups across R1 and R2.

In [ ]:
# Pull the read groups from R1 and R2. Make a concensus read group list.
r1_read_groups_df = r1_df[['read_group']]
r2_read_groups_df = r2_df[['read_group']]

consensus_read_groups_df = r1_read_groups_df.merge(r2_read_groups_df, on ='read_group', how='inner')

In [ ]:
# Quantify obvious pair failures.

r1_N_attempted_read_groups = r1_df.read_group.shape[0]
r2_N_attempted_read_groups = r1_df.read_group.shape[0]
consensus_N_read_groups = consensus_read_groups_df.shape[0]
r1_pcnt_consensus = (consensus_N_read_groups/r1_N_attempted_read_groups)*100
r2_pcnt_consensus = (consensus_N_read_groups/r2_N_attempted_read_groups)*100

print("#"*46)
print(f"R1 had {r1_N_attempted_read_groups} potential read groups, of these {r1_pcnt_consensus} % were among the concensus read groups. R2 had {r2_N_attempted_read_groups} potential read groups, of these {r2_pcnt_consensus} % were among the concensus read groups. In total there are {consensus_N_read_groups} read groups that are matching across R1 and R2 for this experiment.")
print("#"*46)


##############################################
R1 had 10000 potential read groups, of these 100.0 % were among the concensus read groups. R2 had 10000 potential read groups, of these 100.0 % were among the concensus read groups. In total there are 10000 read groups that are matching across R1 and R2 for this experiment.
##############################################


# 3. Reduce the datasets to the read groups that match in R1 and R2.


In [ ]:
# Reduce R1 and R2 to only those in the concensus_read_groups df.

consensus_read_list = consensus_read_groups_df.read_group.unique()

r1_df['in_consensus'] = r1_df.read_group.isin(consensus_read_list)
r2_df['in_consensus'] = r2_df.read_group.isin(consensus_read_list)

r1_reduced_df = r1_df[r1_df['in_consensus'] == True]
r2_reduced_df = r2_df[r1_df['in_consensus'] == True]



# 4. Split into 'hits.\*'  and 'recombinants.\*'.  
'hits.\*' denotes read group matches and the protospacers match.
'recombinants.\*' denotes read group matches but one or more protospacers does not.

In [ ]:
# Get the guide seqs, these relate to the 19 BP segments in the guide_df. Then make its expected complement.

r1_reduced_df['guide_seq'] = [x[1:20] for x in r1_reduced_df.seq]
r2_reduced_df['guide_seq'] = [x[0:19] for x in r2_reduced_df.seq]



In [ ]:
# Build dataset that is used to check for recombination w/in each read group.

r1_guide_read_df = r1_reduced_df[['read_group', 'guide_seq']]
r1_guide_read_df.rename(columns={'guide_seq':'r1_guide_seq'}, inplace=True)

r2_guide_read_df = r2_reduced_df[['read_group', 'guide_seq']]
r2_guide_read_df.rename(columns={'guide_seq':'r2_guide_seq'}, inplace=True)

combined_guide_read_df = r1_guide_read_df.merge(r2_guide_read_df, on='read_group', how='inner')
combined_guide_read_df['combined_guide_seqs'] = combined_guide_read_df['r1_guide_seq'] + "_" + combined_guide_read_df['r2_guide_seq']

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
r1_guide_read_df

,read_group,r1_guide_seq
0,A01256:51:HF2GTDRXY:1:2101:3043:1000,GCGGCGGCGGAGCCTTCGG
1,A01256:51:HF2GTDRXY:1:2101:3495:1000,CCGGGCGCGACGGTCTCGG
2,A01256:51:HF2GTDRXY:1:2101:8612:1000,GCGAGGGTGGAAGATGCGG
3,A01256:51:HF2GTDRXY:1:2101:13892:1000,AAGCCGGTCAGACAGAGGC
4,A01256:51:HF2GTDRXY:1:2101:14488:1000,ACCCGCGCCGTGGTCCCGG
...,...,...
9995,A01256:51:HF2GTDRXY:1:2101:1624:11725,GGCAGAACGCACTGCGAAG
9996,A01256:51:HF2GTDRXY:1:2101:1787:11725,GCAGCTCCGAGGTCGGCGG
9997,A01256:51:HF2GTDRXY:1:2101:1805:11725,GCAGCTCCGAGGTCGGCGG
9998,A01256:51:HF2GTDRXY:1:2101:3052:11725,AGCCAGACTATCTATGTGA


In [ ]:
# Flag expected pairs from the guides_df.

reference_list = guides_df['r1_r2_key'].tolist()
uppercase_reference_list = [x.upper() for x in reference_list]

combined_guide_read_df['uppercase_combined_guide_seqs'] = combined_guide_read_df['combined_guide_seqs'].str.upper()

combined_guide_read_df['non_recombinant'] = combined_guide_read_df['uppercase_combined_guide_seqs'].isin(uppercase_reference_list)

try:
    on_target = combined_guide_read_df['non_recombinant'].value_counts()[1]
except KeyError:
    print("WARNING: There are no on-target hits. Something is probably wrong.")
    on_target = 0
try:  
    recombinant = combined_guide_read_df['non_recombinant'].value_counts()[0]
except KeyError:
    print("WARNING: There are no recombinant hits. Something is probably wrong.")
    recombinant = 0
total_reads = on_target + recombinant
on_target_pcnt = (on_target/total_reads)*100
recombinant_pcnt = (recombinant/total_reads)*100

print("#"*46)
print(f"There are a total of {total_reads} potential read groups after filtering, of these {on_target_pcnt} % were on target for R1 and R2. This means {recombinant_pcnt} % are recombinant read groups.")
print("#"*46)

##############################################
There are a total of 10000 potential read groups after filtering, of these 62.370000000000005 % were on target for R1 and R2. This means 37.63 % are recombinant read groups.
##############################################


In [ ]:
combined_guide_read_df[['read_group']].dtypes

read_group    object
dtype: object

In [ ]:
# Split data into recombinant and hit subsets.

hits_df = combined_guide_read_df[combined_guide_read_df['non_recombinant'] == True][['read_group']]
recombinant_df = combined_guide_read_df[combined_guide_read_df['non_recombinant'] == False][['read_group']]

hits_list = hits_df.read_group.tolist()

r1_reduced_df['hit'] = r1_reduced_df['read_group'].isin(hits_list)
r2_reduced_df['hit'] = r2_reduced_df['read_group'].isin(hits_list)

r1_hits_df = r1_reduced_df[r1_reduced_df['hit'] == True]
r1_recombinant_df = r1_reduced_df[r1_reduced_df['hit'] == False]

r2_hits_df = r2_reduced_df[r2_reduced_df['hit'] == True]
r2_recombinant_df = r2_reduced_df[r2_reduced_df['hit'] == False]

# 5. Export 'hits.\*' and 'recombinants.\*' per fastq.


In [ ]:
# Now its just back to the fastqs from here ... ouch. Start stacking the hits!

r1_hits_stacked_df = r1_hits_df[['title', 'seq', 'plus', 'qual']].stack()
r2_hits_stacked_df = r2_hits_df[['title', 'seq', 'plus', 'qual']].stack()

r1_hits_fastq_df = r1_hits_stacked_df
r2_hits_fastq_df = r2_hits_stacked_df

In [ ]:
# Identify fails, these are recombinants with either R1 or R2 not in the guide list. If the uppercase guide sequences are not in a match to uppercase r1_key or r2_key from guides_df.
uppercase_r1_keys = [x.upper() for x in guides_df['r1_key']]
uppercase_r2_keys = [x.upper() for x in guides_df['r2_key']]

r1_recombinant_df['uppercase_guide_seq'] = [x.upper() for x in r1_recombinant_df['guide_seq']]
r2_recombinant_df['uppercase_guide_seq'] = [x.upper() for x in r2_recombinant_df['guide_seq']]

r1_recombinant_df['in_guide_library'] = r1_recombinant_df['uppercase_guide_seq'].isin(uppercase_r1_keys)
r2_recombinant_df['in_guide_library'] = r2_recombinant_df['uppercase_guide_seq'].isin(uppercase_r2_keys)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [ ]:
# Split R1 and R2 into true recombinants and failed recombinants based on the 'in_guide_library' flag. But first make a list of read groups that fail. Then pull these readgroups to split true recombinants and fails.
r1_failed_recombinants = r1_recombinant_df[r1_recombinant_df['in_guide_library'] == False]
r2_failed_recombinants = r2_recombinant_df[r2_recombinant_df['in_guide_library'] == False]

recombinant_failed_readgroups = r1_failed_recombinants['read_group'].append(r1_failed_recombinants['read_group']).unique()
N_big_fails = len(recombinant_failed_readgroups)

r1_recombinant_df['big_fail'] = r1_recombinant_df['read_group'].isin(recombinant_failed_readgroups)
r2_recombinant_df['big_fail'] = r2_recombinant_df['read_group'].isin(recombinant_failed_readgroups)

r1_failed_recombinants_df = r1_recombinant_df[r1_recombinant_df['big_fail'] == True]
r2_failed_recombinants_df = r2_recombinant_df[r2_recombinant_df['big_fail'] == True]
r1_true_recombinants_df = r1_recombinant_df[r1_recombinant_df['big_fail'] == False]
r2_true_recombinants_df = r2_recombinant_df[r2_recombinant_df['big_fail'] == False]

big_fail_pcnt = (N_big_fails/recombinant)*100

print("#"*46)
print(f"Of the {recombinant} recombinant read groups, {N_big_fails} read groups had a sequence not inthe guide list, so {big_fail_pcnt} % of recombinants can be considered failures.")
print("#"*46)


##############################################
Of the 3763 recombinant read groups, 470 read groups had a sequence not inthe guide list, so 12.490034546904067 % of recombinants can be considered failures.
##############################################


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#new lines: swap sequences
def swap_r1_r2_seq(r2_sequence):
    row = guides_df[guides_df['r2_key'] == r2_sequence[0:19].upper()]
    return row.iloc[0]['r1_key']
# Start stacking the recombinants!
r1_failed_recombinant_stacked_df = r1_failed_recombinants_df[['title', 'seq', 'plus', 'qual']].stack()
r2_failed_recombinant_stacked_df = r2_failed_recombinants_df[['title', 'seq', 'plus', 'qual']].stack()

r1_failed_recombinant_fastq_df = r1_failed_recombinant_stacked_df
r2_failed_recombinant_fastq_df = r2_failed_recombinant_stacked_df

r1_true_recombinant_stacked_df = r1_true_recombinants_df[['title', 'seq', 'plus', 'qual']].stack()
#new line below
r2_true_recombinants_df['seq'] = r2_true_recombinants_df['seq'].map(lambda x: swap_r1_r2_seq(x))
r2_true_recombinant_stacked_df = r2_true_recombinants_df[['title', 'seq', 'plus', 'qual']].stack()

r1_true_recombinant_fastq_df = r1_true_recombinant_stacked_df
r2_true_recombinant_fastq_df = r2_true_recombinant_stacked_df

In [ ]:
# Export the new fastq.

r1_hits_out_file = "hits." + r1_file
r2_hits_out_file = "hits." + r2_file

r1_hits_fastq_df.to_csv(r1_hits_out_file, sep='\t', index=False, header=False, compression='gzip')
r2_hits_fastq_df.to_csv(r2_hits_out_file, sep='\t', index=False, header=False, compression='gzip')

r1_true_recombinant_out_file = "recombinants." + r1_file
r2_true_recombinant_out_file = "recombinants." + r2_file

r1_true_recombinant_fastq_df.to_csv(r1_true_recombinant_out_file, sep='\t', index=False, header=False, compression='gzip')
r2_true_recombinant_fastq_df.to_csv(r2_true_recombinant_out_file, sep='\t', index=False, header=False, compression='gzip')

r1_failed_recombinant_out_file = "fails." + r1_file
r2_failed_recombinant_out_file = "fails." + r2_file

r1_failed_recombinant_fastq_df.to_csv(r1_failed_recombinant_out_file, sep='\t', index=False, header=False, compression='gzip')
r2_failed_recombinant_fastq_df.to_csv(r2_failed_recombinant_out_file, sep='\t', index=False, header=False, compression='gzip')

# Add some narrative.


In [ ]:
print("#"*46)
print("Your analysis has just finished.")
print("Reads from matched read groups on whose guides were on target for both R1 and R2 are found in the files prefixed hits.*.")
print("Reads from matched read groups on whose guides were on were off target and considered recombinant for either R1 and R2 are found in the files prefixed recombinants.*.")
print("Reads from recombinant read groups on whose guides were not mathced to a known guide sequence for either R1 and R2 are found in the files prefixed fails.*.")
print("Good luck and feel free to generally ignore any outputs below here!")
print("#"*46)

##############################################
Your analysis has just finished.
Reads from matched read groups on whose guides were on target for both R1 and R2 are found in the files prefixed hits.*.
Reads from matched read groups on whose guides were on were off target and considered recombinant for either R1 and R2 are found in the files prefixed recombinants.*.
Reads from recombinant read groups on whose guides were not mathced to a known guide sequence for either R1 and R2 are found in the files prefixed fails.*.
Good luck and feel free to generally ignore any outputs below here!
##############################################


# Just for executable testing below.

In [ ]:
%%bash
# python 19bp_dual_guide_parser_tool.py --help
python 19bp_dual_guide_parser_tool.py --guides_file 20200513_library_1_2_unbalanced_dJR051.csv --r1_file UDP0007_S1_R1_001.fastq.gz --r2_file UDP0007_S1_R2_001.fastq.gz --N_reads 100000

##############################################
Thanks for trying the dual_guide_parser from CARD + iNDI + DTi.
To run this code, you will need to specify the guides_file, this is a file similar to the example 20200513_library_1_2_unbalanced_dJR051.csv.
You will need to specify a pair of R1 and R2 files, such as UDP0007_S1_R1_001.fastq.gz and UDP0007_S1_R2_001.fastq.gz.
You can also specify the number of read groups you are interested for testing the tool, this relates to the option n_groups. This will only read that many readgroups from the R1 and R2 files, allowing you to speed things up a bit.
This code must be run from the working directory that contains the R1 and R2 files, but the guides_file can be anywhere, just specify a full path to the guides file like ~/Desktop/20200513_library_1_2_unbalanced_dJR051.csv.
This is best run on a large RAM / high CPU set up as the files are quite large.
##############################################
##############################################

19bp_dual_guide_parser_tool.py:40: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
19bp_dual_guide_parser_tool.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r1_read_groups_df['read_group'] = r1_read_groups_df["fastq_content"].str.split(" ", n = 1, expand = True)[0]
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
19bp_dual_guide_parser_tool.py:153: SettingWithCopyWarning: 
A value